# Arrays and Loops

We'll cover:
1. Array literals
2. Concatenation
3. For loops
4. Comprehensions
5. Element types
6. Dequeues
7. The `bang!` convention
8. Variable names vs. copies

# Arrays

Julia has highly efficient multidimensional arrays, both constructed and indexed with square brackets.

Syntax: <br>
```julia
[item1, item2, ...]
```

In [1]:
squares = [1, 4, 9, 15, 25, 36, 49, 64]

8-element Array{Int64,1}:
  1
  4
  9
 15
 25
 36
 49
 64

In [7]:
#Indexing starts from 1 rather than 0
squares[1]

1

In [8]:
squares[1:3]

3-element Array{Int64,1}:
 1
 4
 9

In [9]:
squares[end]

64

In [10]:
squares[4] = 16

16

In [11]:
squares

8-element Array{Int64,1}:
  1
  4
  9
 16
 25
 36
 49
 64

In [12]:
typeof(squares)

Array{Int64,1}

## Concatenation

If, instead of commas, you just use spaces, then the values are concatenated horizontally.

In [13]:
cubes = [1, 8, 27, 64, 125, 216, 343, 512]

8-element Array{Int64,1}:
   1
   8
  27
  64
 125
 216
 343
 512

In [14]:
powers = [1:8 squares cubes]

8×3 Array{Int64,2}:
 1   1    1
 2   4    8
 3   9   27
 4  16   64
 5  25  125
 6  36  216
 7  49  343
 8  64  512

In [16]:
# Row 4 column 2
powers[4, 2]

16

In [17]:
powers[:, 3]

8-element Array{Int64,1}:
   1
   8
  27
  64
 125
 216
 343
 512

In [18]:
powers[7, :]

3-element Array{Int64,1}:
   7
  49
 343

In [19]:
typeof(powers)

Array{Int64,2}

Semicolon separators perform vertical concatenation:

In [20]:
[squares; cubes]

16-element Array{Int64,1}:
   1
   4
   9
  16
  25
  36
  49
  64
   1
   8
  27
  64
 125
 216
 343
 512

Whereas commas would simply create an array of arrays:

In [21]:
nested_powers = [[1,2,3,4,5,6,7,8], squares, cubes]

3-element Array{Array{Int64,1},1}:
 [1, 2, 3, 4, 5, 6, 7, 8]          
 [1, 4, 9, 16, 25, 36, 49, 64]     
 [1, 8, 27, 64, 125, 216, 343, 512]

In [22]:
[1,2,3,4,5,6,7,8, squares, cubes]

10-element Array{Any,1}:
 1                                  
 2                                  
 3                                  
 4                                  
 5                                  
 6                                  
 7                                  
 8                                  
  [1, 4, 9, 16, 25, 36, 49, 64]     
  [1, 8, 27, 64, 125, 216, 343, 512]

In [23]:
nested_powers[2]

8-element Array{Int64,1}:
  1
  4
  9
 16
 25
 36
 49
 64

Horizontal and vertical concatenation can be used together to as a simple syntax for matrix literals:

In [24]:
[1 3 5; 2 4 6]

2×3 Array{Int64,2}:
 1  3  5
 2  4  6

# Loops

Of course, we could construct this programmatically with a for-loop. The syntax for a `for` loop is

```julia
for *var* in *loop iterable*
    *loop body*
end
```



In [25]:
A = fill(0, (8, 3)) # Allocate an 8x3 matrix to store the values into
for pow in 1:3
    for value in 1:8
        A[value, pow] = value ^ pow
    end
end
A

8×3 Array{Int64,2}:
 1   1    1
 2   4    8
 3   9   27
 4  16   64
 5  25  125
 6  36  216
 7  49  343
 8  64  512

In [26]:
A == powers

true

## Array Comprehensions

In [27]:
squares = [value^2 for value in 1:8]

8-element Array{Int64,1}:
  1
  4
  9
 16
 25
 36
 49
 64

In [28]:
cubes = [value^3 for value in 1:8]

8-element Array{Int64,1}:
   1
   8
  27
  64
 125
 216
 343
 512

In [29]:
powers = [value^pow for value in 1:8, pow in 1:3]

8×3 Array{Int64,2}:
 1   1    1
 2   4    8
 3   9   27
 4  16   64
 5  25  125
 6  36  216
 7  49  343
 8  64  512

# The element type

Note that every time an array prints out, it is displaying its element type and dimensionality, for example `Array{Int64, 2}`. This describes what it can store — and thus what it can return upon indexing.

In [30]:
typeof(powers)

Array{Int64,2}

In [31]:
typeof(powers[1, 1])

Int64

Further, the array will try to convert any new values assigned into it to its element type:

In [34]:
# Lossless conversion not possible
powers[1, 1] = 1.6

InexactError: InexactError: Int64(Int64, 1.6)

In [35]:
Int64(1.6)

InexactError: InexactError: Int64(Int64, 1.6)

In [33]:
powers[1, 1] = -5.0 # This can be losslessly converted to an integer

-5.0

In [36]:
powers

8×3 Array{Int64,2}:
 -5   1    1
  2   4    8
  3   9   27
  4  16   64
  5  25  125
  6  36  216
  7  49  343
  8  64  512

Arrays that have an exact and concrete element type are generally significantly faster, so Julia will try to find an amenable element type for you in its literal construction syntax:

In [37]:
fortytwosarray = [42, 42.0, 4.20e1, 4.20f1, 84//2, 0x2a]

6-element Array{Float64,1}:
 42.0
 42.0
 42.0
 42.0
 42.0
 42.0

In [38]:
for x in fortytwosarray
    show(x)
    println("\tisa $(typeof(x))")
end

42.0	isa Float64
42.0	isa Float64
42.0	isa Float64
42.0	isa Float64
42.0	isa Float64
42.0	isa Float64


The `Any` array can be helpful for disabling these behaviors and allowing all kinds of different objects:

In [39]:
anyfortytwos = Any[42, 42.0, 4.20e1, 4.20f1, 84//2, 0x2a]

6-element Array{Any,1}:
   42    
   42.0  
   42.0  
   42.0f0
  42//1  
 0x2a    

In [40]:
anyfortytwos[1] = "FORTY TWO"
anyfortytwos

6-element Array{Any,1}:
     "FORTY TWO"
   42.0         
   42.0         
   42.0f0       
  42//1         
 0x2a           

# Vectors as dequeues

One-dimensional arrays can be appended to and have items removed from them:

In [41]:
fib = [1, 1, 2, 3, 5, 8, 13]

7-element Array{Int64,1}:
  1
  1
  2
  3
  5
  8
 13

In [42]:
push!(fib, 21)

8-element Array{Int64,1}:
  1
  1
  2
  3
  5
  8
 13
 21

In [44]:
# Append sum of last two elements to *fib* vector - run twice
push!(fib, sum(fib[end-1:end]))

10-element Array{Int64,1}:
  1
  1
  2
  3
  5
  8
 13
 21
 34
 55

In [45]:
pop!(fib)

55

In [46]:
# Push in beginning
pushfirst!(fib, 0)

10-element Array{Int64,1}:
  0
  1
  1
  2
  3
  5
  8
 13
 21
 34

In [47]:
popfirst!(fib)

0

## Aside: why so shouty!?

Why are there exclamations in all the above function calls? `push!(fib, ...)`, `pop!(fib)`, etc.?

In [48]:
push!

push! (generic function with 22 methods)

This is entirely a convention. It's a signal to the caller that one of the argument is going to be _mutated_. It's perhaps easiest to demonstrate with an example:

In [49]:
A = rand(0:10, 10)

10-element Array{Int64,1}:
 8
 8
 3
 3
 0
 3
 7
 1
 8
 3

In [50]:
sort(A)

10-element Array{Int64,1}:
 0
 1
 3
 3
 3
 3
 7
 8
 8
 8

In [51]:
A

10-element Array{Int64,1}:
 8
 8
 3
 3
 0
 3
 7
 1
 8
 3

In [52]:
sort!(A)

10-element Array{Int64,1}:
 0
 1
 3
 3
 3
 3
 7
 8
 8
 8

In [53]:
A # That changed A!

10-element Array{Int64,1}:
 0
 1
 3
 3
 3
 3
 7
 8
 8
 8

## Aside: names vs. copies

Remember that variables are just names we give our objects. So watch what happens if we give the same object two different names:

In [54]:
fibonacci = [1, 1, 2, 3, 5, 8, 13]

7-element Array{Int64,1}:
  1
  1
  2
  3
  5
  8
 13

In [55]:
some_numbers = fibonacci
some_numbers[1] = 404
some_numbers

7-element Array{Int64,1}:
 404
   1
   2
   3
   5
   8
  13

In [56]:
fibonacci

7-element Array{Int64,1}:
 404
   1
   2
   3
   5
   8
  13

In [57]:
fibonacci[1] = 1
some_numbers = copy(fibonacci)
some_numbers[2] = 404
fibonacci

7-element Array{Int64,1}:
  1
  1
  2
  3
  5
  8
 13

In [58]:
some_numbers

7-element Array{Int64,1}:
   1
 404
   2
   3
   5
   8
  13